In [1]:
import numpy as np
from scipy import optimize, stats

### Stochastic Inventory Problem for Discrete Demand

In [ ]:
def backorder_loss(q, x, prob, p_cost):
    penalty = p_cost*(val[0, (x+q):] - x - q).dot(prob[0, (x+q):])
    return penalty

def der_backorder_loss(q, x, val, prob, p_cost):
    interval = np.where(val[0, :] > x + q)
    der_penalty = p_cost*np.sum(prob[0, interval])
    return der_penalty

def holding_loss(q, x, val, prob, h_cost, tol=1e-6):
    holding = 0
    for mean, var in zip(np.cumsum(d_mean), np.cumsum(np.square(d_std))):
        dist = stats.norm(mean, np.sqrt(var))
        held = dist.expect(lambda u: h_cost*(x + q - u), lb=x, ub=(x+q))
        if held < tol:
            break
        holding += held
    return holding

def der_holding_loss(q, x, d_mean, d_std, h_cost, tol=1e-6):
    der_holding = 0
    for mean, var in zip(np.cumsum(d_mean), np.cumsum(np.square(d_std))):
        dist = stats.norm(mean, np.sqrt(var))
        held = dist.cdf(x + q) - dist.cdf(x)
        der_holding += h_cost*held
    return der_holding

def sicp_root(q, x, d_mean, d_std, h, p):
    return holding_loss(q, x, d_mean, d_std, h) \
        - backorder_loss(q, x, d_mean, d_std, p)

def sicp_root_der(q, x, d_mean, d_std, h, p):
    return der_holding_loss(q, x, d_mean, d_std, h) \
        + der_backorder_loss(q, x, d_mean, d_std, p)

def sicp_max(q, x, d_mean, d_std, h, p):
    return max(holding_loss(q, x, d_mean, d_std, h),
               backorder_loss(q, x, d_mean, d_std, p))